$$ \huge \lambda \quad Calculus $$

This code is explained in, and is the substance of, Chapter 4 of the report: $$\lambda\text{_Calculus.pdf}$$ found in the Github Repo.



In [41]:
# Here are the famous functions S, K and I, written in lambda form

S = lambda x: lambda y: lambda z: x(z)(y(z))
K = lambda x: lambda y: x # == TRUE
I = lambda x: x # == ID

--------------------------------------------------------------------------------------------------------------------------------

$$ \large Function \quad Trees \quad - \quad Method \quad 1$$

In [45]:
import copy
from itertools import product
from itertools import permutations
import numpy as np

def tree_builder(di, order):
    '''
    Generates a function tree as a list of the locations of branch points at each level. See final report sections
    4.1, 4.2.1 and 4.2.2.
    
    Input: The drawing code (see section 4.2.1) for the tree as two lists: di, order.
    
    Output: An outer list containing inner lists, each inner list represents a level of the function tree,
    each pair of integers represents the indices of the branches meeting.
    
    '''
    
    # make copies of the input
    di1 = di.copy()
    order1 = order.copy()
    
    # initialise tree outer list
    tree = []
    
    # find the number of functions ('leaves') and define such a list
    n = len(di1)
    fs = list(np.linspace(0, n-1, n, dtype=int))
    
    # while we have more than one connection to be made...
    while len(di1) > 2:

        lvl = []
        branch = []

        for i, d in enumerate(di1):
            
            # ... find the branches pointing at eachother and log the connection
            if d == "L" and di1[i-1] == "R":
                lvl.append([i-1, i])
                branch.append([fs[i-1], fs[i]])

        # The following code updates the list of directions and orders to reflect...
        # ...the tree "a level above", after the identified connections
        
            to_remove = []

        for i, l in enumerate(lvl):

            k = l[0]

            if order1[k] < order1[k+1]:
                to_remove.append(k+1)

            elif order1[k] > order1[k+1]:
                to_remove.append(k)
        
        j = 0
        for i in range(len(di1)):
            if i in to_remove:
                del di1[i-j]
                del order1[i-j]
                del fs[i-j]
                j += 1
        # Two connected branches at an earlier level represented in the level above by "L" and... 
        # ...the order number of the branch drawn first
        
        tree.append(branch)
        
        
        # renumber the functions after removing the connected branches
        fs = list(np.linspace(0, n-1-len(to_remove), n-len(to_remove), dtype=int))
    
    tree.append([[0, 1]])
    return tree

In [46]:
def tree_to_exp(tree, n):
    '''
    Takes a tree generated by tree_builder and generates the star expression (see section 4.1 - Intro and Star form).
    
    Effectively "reads" the tree in list form and writes the star form of the corresponding functional composition
    as a string.
    
    Input: tree list formed by tree_builder
    Output: corresponding functional composition in star form
    '''

    f = list(range(0, n))
    f = [str(i) for i in f]
    
    f2 = [i for i in list(range(0, n))]

    exps = [f, ]
    nums = [f2, ]

    for i in range(len(tree)):

        exp = []
        num = []
    
        j = 0
        while j < len(f): # while there are branch points to be found...
            
            # find them and represent them in string and star form
            if [j, j+1] in tree[i]:
                term = f"({f[j]} * {f[j+1]})"
                exp.append(term)
                
                no = [f2[j], f2[j+1]]
                num.append(no)
                j += 2
            
            # branches not connected are brought forward to the next iteration
            else:
                term = f"{f[j]}"
                exp.append(term)
                
                no = f2[j]
                num.append(no)
                j += 1
                
        
        # We append the star form expression with thus far connections
        exps.append(exp)
        nums.append(num)
        # update the branches, i.e. two branches "1" and "2", when connected, are now one branch called "(1 * 2)"
        f = exp
        f2 = num

    return exps[-1][0] # return full expression

## Catalan Numbers

The following code generates a the full list of n potential drawing codes, then uses our predefined functions to generate the trees. We then take away duplicates and count how many distinct trees we have. As it turns out, __the number of distinct trees containing n functions__ is $$\mathbb{C}_{n-1}$$, __the (n-1)th Catalan number__ (See section 4.2).

In [ ]:
times2 = []
# n = 2

list_lens = []

tree_list = []
exp_list = []
ns = []

for n in range(2, 10):
    time1 = time.time()
    tree_sublist = []
    
    order = np.linspace(1, n-2, n-2, dtype=int)
    di = ["L", "R"]
    dis = [p for p in product(di, repeat=n-2)]
    orders = [p for p in permutations(order)]

    for i, order in enumerate(orders):
        orders[i] = [0] + list(order) + [0]

    for j, di in enumerate(dis):
        dis[j] = ["R"] + list(di) + ["L"]
        
    for order in orders:
        for di in dis:
            tree = tree_builder(di, order)
            exp = tree_to_exp(tree, n)
            
            if tree not in tree_list:
                tree_list.append(tree)
                
            if tree not in tree_sublist:
                tree_sublist.append(tree)
                ns.append(n)
                
            exp_list.append(exp)
        
    list_lens.append(len(tree_sublist))
    
    exp_list = list(set(exp_list))
    
    time2 = time.time()
    time3 = time2 - time1
    times2.append(time3)
    
    print(n, " - ", len(exp_list))


print(list_lens)
print("THE CATALAN NUMBERS !!!!!")
print(times2)

In [47]:
# drawcode times
# [0.03536558151245117, 0.0020029544830322266, 0.005005598068237305, 0.02970743179321289, 
#  0.21956300735473633, 1.329827070236206, 19.30217432975769, 259.8391442298889]

In [48]:
# Print to see the Catalan numbers:

import math

for n in range(1, 11):
    print(f"n={n} -> ", int(math.factorial(2*n) / (math.factorial(n+1) * math.factorial(n))))

n=1 ->  1
n=2 ->  2
n=3 ->  5
n=4 ->  14
n=5 ->  42
n=6 ->  132
n=7 ->  429
n=8 ->  1430
n=9 ->  4862
n=10 ->  16796


--------------------------------------------------------------------------------------------------------------------------------

$$ \large Function \quad Trees \quad - \quad Method \quad 2$$
Find detailed explanation for the motivation for the code below is section 4.2.2.

In [52]:
def find_combos(arr, index, num, reducing_sum, arrs):
    '''
    This function, in tandem with its driverfunction get_combos, generates a list 
    of all integers which add up to the integer: num

    Inputs:

    arr: a list for storing integers
    arrs: list for storing full combinations
    index: integer for logging how many times function has been previously called
    num: int we would to sum to
    reducing_sum: counter to keep track how close to num we are as we call recursively
    '''
    if reducing_sum < 0: # Make sure we stop after we sum to n
        return

    # Store combination when found
    if reducing_sum == 0:
        arrs.append(arr[:index])
        return

    # Find the number the previous combination started with (to maintain an increasing order)
    if index == 0:
        prev = 1
    else:
        prev = arr[index - 1]

    # We loop starting from that previous number
    for k in range(prev, num + 1):

        # We add the next element to the list summing up to num
        arr[index] = k

        # We call recursively with the, logging how close we are to num and how many integers we have added
        find_combos(arr, index + 1, num, reducing_sum - k, arrs)

In [53]:
def get_combos(n, arrs):
    '''
    Driver function for find_combos()
    '''
     
    # array to store the combinations: can contain max n elements
    arr = [0] * n
 
    find_combos(arr, 0, n, n, arrs)

In [57]:
arrs = []

get_combos(4, arrs)
print(arrs)

[[1, 1, 1, 1], [1, 1, 2], [1, 3], [2, 2], [4]]


In [68]:
import math
from itertools import permutations

def next_levels(canopy):
    '''
    For a full and detailed explanation of what motivates this code, see section 4.2.2.
    
    Takes a level of branch counts and gives every valid next level, by 
    Definition 8: Axioms of Validity for Branch-count Pyramids, found at the top of page 31.
    
    Makes use of get_combos.
    '''
    
    n = sum(canopy) # identify total number of leaves on the tree
    width = len(canopy) # how many unconnected branches there are
    
    if width < 3:
        return [n] # if only two branches are to be connected (top of the pyramid)
    
    arrss = []
    get_combos(n, arrss) # find all the combos which add up to n
    
    arrs = [arr for arr in arrss if width > len(arr) >= math.ceil(width/2)] # disregards combos of impossible length
    arrs = [list(permutations(a)) for a in arrs] # find all permutations of remaining combos
    
    list_sum = [a for arr in arrs for a in arr] # remove innner lists, dispense into one larger one
    arrs = list_sum
    arrs = list(set(arrs)) # remove duplicates
    
    # Here we remove the remaining invalid permutations, by the axioms on page 31.
    valid_arrs = []
    for arr in arrs:
    
        j = 0
        i = 0
        valid = True
        
        while j < width:
            j += 1
            i += 1
            
            # If we have reached the end of the row, thus far valid, and branch counts do not match up 
            # it must be invalid
            if canopy[j-1] != arr[i-1] and j == width:
                valid = False 
                break
            
            # If branch counts don't match
            if canopy[j-1] != arr[i-1]:
                if canopy[j-1] + canopy[j] == arr[i-1]: # it might be because of a connection...
                    j += 1
                else:
                    valid = False # ...or its invalid
                    break
        
        if valid == True:
            valid_arrs.append(arr) # valid permutations get saved
        
    return valid_arrs

In [69]:
def get_pyramids(Z, tree_list):
    '''
    Uses next_levels() to create a list of all possible branch count pyramids of leaf number n.
    
    Inputs:
    
    Z = the list [[1, 1, 1, ... {n times}, 1]], i.e. a selection of n leaves from which the function produces 
    all possible trees
    
    tree_list = empty list to be filled
    
    
    Output: a list of lists, each inner list a possible, distinct tree.
    
    '''

    lvls = next_levels(Z[-1]) # Finds possible next levels

    array = [Z.copy() for i in range(len(lvls))] # copies Z that many times ...
    
    for i in range(len(lvls)):
        array[i].append(lvls[i]) # ...and adds each possibility to its Z copy
        
    for arr in array:
        if type(arr[-1]) == int:
            tree_list.append(arr) # If you've found a complete tree, save it
        else:
            get_pyramids(arr, tree_list) # else recursively call the function again

In [70]:
def tree_formation(tree):
    '''
    Takes a branch count pyramid, or a function tree expressed in branch count form, and gives the tree in 'tree form'
    (as described in section 4.1).
    '''
    big_tree = []
    
    # The code below scans along a tree in branch count form, when it identifies a connection
    # it adds it to the tree form expression
    for i in range(len(tree)-2):
        sub_tree = []
        j = -1
        k = -1
        while j < len(tree[i])-2:
            j += 1
            k += 1
            if tree[i][j] != tree[i + 1][k]:
                if tree[i][j] + tree[i][j + 1] == tree[i + 1][k]:
                    sub_tree.append([j, j+1])
                    j += 1
        
        big_tree.append(sub_tree) # at each level records the connection
    
    big_tree.append([[0, 1]]) # the final connection
    return big_tree

## Finding all the distinct functional compositions (trees) (again).
This next code block uses all the functions above to generate (again) all the distinct trees. Gratifyingly, we get once again the Catalan numbers. To read a good motivation for this bijection, see section 4.2.3 of the report.

In [104]:
import time
times = []

lastones = 0

for n in range(2, 7):
    
    exp_list = []
    
    time1 = time.time()
    
    Z = [[1]]
    for i in range(n-1):
        Z[0] += [1]
    tree_list = []
    get_pyramids(Z, tree_list)

    new_tree_list = []

    for tree in tree_list:
        new_tree = tree_formation(tree)
        new_tree_list.append(new_tree)
    
    for tree in new_tree_list:
        exp = tree_to_exp(tree, n)
        exp_list.append(exp)
    
    time2 = time.time()
    time3 = time2 - time1
    times.append(time3)
    
    print(len(list(set(exp_list))))
print("...and so on")

1
2
5
14
42
...and so on


In [90]:
print(times)

[]


In [ ]:
# branch count generates with duplicates
# 1, 2, 7, 34, 214, 1652, 15121, 160110, 1925442

In [98]:
import numpy as np

# Comparing the number of duplicates (and thus the efficiency) of the two methods of generating every possible composition

branchtimes = np.array([2.8836843967437744, 0.003603696823120117, 0.0011317729949951172, 0.004877805709838867, 
               0.03816676139831543, 0.3340425491333008, 5.647143840789795, 43.592026472091675])

drawtimes = np.array([0.03536558151245117, 0.0020029544830322266, 0.005005598068237305, 0.02970743179321289, 
             0.21956300735473633, 1.329827070236206, 19.30217432975769, 259.8391442298889])

branchcounts = np.array([1, 2, 7, 34, 214, 1652, 15121, 160110, 1925442])

drawcounts = np.array([1, 2, 8, 48, 384, 3840, 46080, 645120, 10321920])

ns = np.linspace(2, 10, 8)

In [ ]:
# branch count times
# [2.8836843967437744, 0.003603696823120117, 0.0011317729949951172, 0.004877805709838867, 
#  0.03816676139831543, 0.3340425491333008, 5.647143840789795, 43.592026472091675]

In [ ]:
# 1, 2, 6, 30, 200, 1594, 14805, 157884, 1906185, ...

-------------------------------------------------------------------------------------------------------------------------------


$$ \large SKI \quad Combinatorics $$

The following code is relevant to section 4.3: "SKI Combinatorics".

In [109]:
def find_input(multiplier, exp):
    '''
    find_input() is a very important function to our project. It is a parser, which scans a functional composition
    in star form (see section 4.1) and, when given a function within the composition, returns the function it is 
    itself acting upon.
    
    For example, find_input(multiplier = "1", exp = "(1 * (2 * 3))") returns "(2 * 3)".
    '''

    right = 0
    left= 0
    inputs = False
    number = False

    leng = len(multiplier)

    inp = ""
    num = ""

    for i, char in enumerate(exp):

        if exp[i-4-leng:i] == "(" + multiplier + " * ": # find the multiplier in the exp
            inputs = True # begin scanning for input
            right = 0 # brackets are balanced at zero
            left = 0
            
        # if we are finding numbers initially, this must be the input (but could be multiple digits, like 12)
        if inputs and left == 0 and char.isnumeric():                      
            number = True
            num += char

        if number and left == 0 and not char.isnumeric():
            inp = num
            return inp # if we have been storing a number input, and the next character is not numeric, 
                       # we return what we have

        # we count left and right brackets
        if char == "(":
            left += 1
        if char == ")":
            right += 1

        # while we have more left brackets then right, we must still be scanning over input
        if inputs and left >= right:
            inp += char

        if inputs and not number and left == right:
            left = 0
            right = 0
            return inp # when brackets are balanced again, we return what we have

In [123]:
def get_structure(exp):
    '''
    Splits an expression into its functions (in order) and its composition structure. We need this because find_input
    takes composition structures, with numerical placeholders
    '''
    new_exp = ""
    SKI = []
    j = 0
    for char in exp:
        if char == "S" or char == "K" or char == "I":
            new_exp += f"{j}"
            SKI.append(char)
            j += 1
        else:
            new_exp += char

    return new_exp, SKI 

In [124]:
import re

def subin_to_exp(exp, SKI):

    l = len(SKI) - 1
    for i, s in enumerate(SKI):
        exp = re.sub(fr"{l - i}", fr"{SKI[l - i]}", exp)
    
    return exp

In [125]:
p = '((((((0 * (((1 * 2) * 3) * 4) * 5) * 6) * 7) * 8) * 9) * (10 * (11 * 12)))'
h = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm']

subin_to_exp(p, h)

'((((((a * (((b * c) * d) * e) * f) * g) * h) * i) * j) * (k * (l * m)))'

In [126]:
### defining SKIsimplify()
import re

def SKIsimplify(SKI_exp):
    
    exp, SKI = get_structure(SKI_exp)
    
    only_S = False
    
    shortest_exp = (len(exp), exp, SKI)
    
    it = 0
    j = -1
    while j < len(SKI)-1:
        it += 1
        j +=1
        
        
        mult = f"{j}"
        # identifying the functions curried inputs
        inputs = []
        while len(inputs) <= 3:
            inp = find_input(mult, exp)
            if inp == None:
                break

            inputs.append(f"{inp}")
            mult = f"({mult} * {inp})"

        ## Create inputss, its just inputs but has a \ before every symbol so re.sub can read it
        inputss = []
        for inp in inputs:
            inps = ""
            for i in inp:
                if i == "(" or i == ")" or i == "*":
                    i = re.sub(fr"\{i}", fr"\\{i}", i) 
                inps += i
            inputss.append(inps)

        
        
        ### Performing the action of the combinators in the expression, see Chapter 3 for an indepth explanation of 
        # the S, K and I combinators, and combinators themselves
        
        if SKI[int(j)] == "K" and len(inputs) >= 2:

            exp = re.sub(fr"\(\({j} \* {inputss[0]}\) \* {inputss[1]}\)", fr"{inputs[0]}", exp)
            j = -1
            only_S = False

        elif SKI[int(j)] == "I" and len(inputs) >= 1:

            exp = re.sub(fr"\({j} \* {inputss[0]}\)", fr"{inputs[0]}", exp)
            j = -1
            only_S = False
                    
        elif SKI[int(j)] == "S" and len(inputs) >= 3 and only_S: 
            # As S is the only one which doesnt REDUCE, we prioritise the action sof the other two

            exp = re.sub(fr"\(\(\({j} \* {inputss[0]}\) \* {inputss[1]}\) \* {inputss[2]}\)", 
                         fr"(({inputs[0]} * {inputs[2]}) * ({inputs[1]} * {inputs[2]}))", exp)
            j = -1
            only_S = False
        
        if j == len(SKI) - 1 and not only_S:
            j = -1
            only_S = True


        ## Renumbering our new expression, and finding the new order of S, K, Is
        nums = []
        new_exp = ""
        count = 0
        num = ""
        number = False
        for i, char in enumerate(exp):
            if char.isnumeric() and len(exp) == 1:
                number = True
                new_exp += "0"
                nums.append(char)
            if char.isnumeric() and len(exp) != 1:
                number = True
                num += char
            if number and not char.isnumeric():
                number = False
                new_exp += f"{count}"
                nums.append(num)
                num = ""
                count += 1
            if not number:
                new_exp += char

        ## Reording our SKI list according to the new combination after the action of the combinators
        new_SKI = []
        for no in nums:
            new_SKI.append(SKI[int(no)])
        
        ## Updating our expression and SKI list
        SKI = new_SKI
        exp = new_exp
        
        ## update the shortest new_exp so far - will be returned if the combo is volatile
        if len(exp) < shortest_exp[0]:
            shortest_exp = (len(exp), exp, SKI)
        if it > 100:
            return subin_to_exp(shortest_exp[1], shortest_exp[2]), "vol"
        
    return subin_to_exp(exp, SKI), "con"

## Finding distinct SKI functions
Indepth explanation of combinators and S, K and I, as well as the motivation behind the code, can be found in Section 4.3. Given the (mind-boggling) nature of combinators (see https://writings.stephenwolfram.com/2020/12/combinators-a-centennial-view/ to have your mind blown) we must reduce them to find out which ones are in fact the same function. 

We cant reduce them directly with Python, for some will recombine forever (or too long), and cause a stackoverflow. What's more we'd be doing it completely blind. Thus we use SKIsimplify() to induce the actions of the combinators. Most reduce to normal form with relatively few iterations, and the others SKIsimplify disregards as "volatile".

The following block takes a long time to run for n close to 10.

In [ ]:
collation_con = np.zeros([2, 11])
collation_vol = np.zeros([2, 11])

SKI_expsss = []

for n in range(2, 6):
    
    volatile = []
    SKI_exps = []
    
    #----------Using the branch count method to find all distinct compositions-------------
    
    exp_list = []
    Z = [[1]]
    for i in range(n-1):
        Z[0] += [1]
    tree_list = []
    get_pyramids(Z, tree_list)

    new_tree_list = []

    for tree in tree_list:
        new_tree = tree_formation(tree)
        new_tree_list.append(new_tree)

    for tree in new_tree_list:
        exp = tree_to_exp(tree, n)
        exp_list.append(exp)

    exp_list = list(set(exp_list))
    #---------------------------------------------------------------------------------------
    #------------We reduce each expression using SKIsimplify--------------------------------
    count = 0
    for i in range(len(exp_list)):

        exp = exp_list[i]

        ski = ["S", "K", "I"] # should I include "I" or not?
        SKIS = [p for p in product(ski, repeat=n)]

        for SKI in SKIS:
            count += 1
            SKI_exp = subin_to_exp(exp, SKI)

            simple_exp, stability = SKIsimplify(SKI_exp)

            if stability == "vol":
                volatile.append(simple_exp)

            if stability == "con":
                SKI_exps.append(simple_exp)
                
    #--------------------------------------------------------------------------------------

# SKI_exps.append("S")
# SKI_exps.append("K")
# SKI_exps.append("I")
    print(count)
    
    # We get the numbers of expressions created/duplicated/normal forms for each n
    collation_con[0][n] = len(SKI_exps)
    collation_vol[0][n] = len(volatile)
    
    SKI_exps = list(set(SKI_exps))
    volatile = list(set(volatile)) # we remove duplicates
    
    collation_con[1][n] = len(SKI_exps)
    collation_vol[1][n] = len(volatile)

#     print(len(SKI_exps) + len(volatile))
    
    SKI_expsss += SKI_exps

In [ ]:
### MAX ITER before labelled volatile == 50:


### [2, 3, 4, 5, 6, 7, 8, 9] ###

#####################################################
# cons After normalizing
# [9, 54, 405, 3402, 30'613, 288'469, 2'808'424, 28'010'829]

# distinct cons
#  [9, 30, 108, 438, 192, 869, 40'584, 192'725]

# vols after normalizing
# [0, 0, 0, 0, 5, 215, 6245, 135'861]
#####################################################

# Distinct vols
#  [0, 0, 0, 0, 5, 159, 3267, 43'334]

In [ ]:
# import json

# # writing the data to file
# with open('SKI_exps.txt', 'w') as file:
#     json.dump(SKI_exps, file)

In [ ]:
import json

with open('SKIdata.txt') as f:
    SKI_data = json.load(f)
    f.close()

print(len(SKI_data))

---------------------------------------------------------------------------------------------------------------------------

$$ \large Finding \quad Functions $$
This code is covered in section 4.4 of the report.

Here we take each of our distinct SKI forms (as strings so we can see them) and test each one to find lambda functions created in Chapter 2 - Booleans and the Integers. We use SKIsimplify to initially try compositions, as we have no way of knowing which combinations will cause a stack overflow.

First we must create a class 'FNCT', which will allow us to evaluate our functional compositions in star form, by translating star form directly into true Python syntax.

In [113]:
class FNCT():
    
    def __init__(self, string):
        self.str = string # the string
        self.f = eval(string) # the pythonic function
    
    def __mul__(self, other, *args): # for members of the FNCT class, "*" now means function eval
        new_str = "(" + self.str + other.str + ")"
        return FNCT(new_str)
    
    def __call__(self, other): # we can use our star forms as functions and give them valid input
        return self.f(other)
        

In [135]:
# Make S, K and I instances of the FNCT class
s = '(lambda x: lambda y: lambda z: x(z)(y(z)))'
k = '(lambda x: lambda y: x)'
i = '(lambda x: x)'
S = FNCT(s)
K = FNCT(k)
I = FNCT(i)

# Thanks to FNCT "((S * K) * K)" can now be evaluated by Python 
SKI = "((S * K) * K)"
print(SKI + "(I)")
print("= ", eval(SKI)("I")) # we can apply it to objects [S(K)(K) is the identity]

print(SKIsimplify("(" + SKI +" * I)")[0]) # SKIsimplify agrees

((S * K) * K)(I)
=  I
I


### Function testing: making shortlists with SKIsimplify()
We must test SKI functions physically (using SKIsimplify) because we have no way of knowing when we will trigger a stack overflow. Thus, we do so and match the output strings to what we would expect to see were it the function we were searching for (be it ONE, TWO, NEXT, OR, TRUE, etc). A few match our initial criteria, and those we test further using our functions defined in Integers.ipynb, and explained in great detail in Chapter 2, posINT_show() and bool_SHOW(). The shortlisted functions we can directly evaluate in Python, having had their convergence confirmed by SKIsimplify.

In [ ]:
maybeones = []
for SKI_exp in SKI_data:
    try:
        
        maybeone = SKIsimplify("("+ SKI_exp + " * (K * I))")[0]
        if maybeone == "I":
            maybeones.append(SKI_exp)
            
    except:
        pass

In [ ]:
### Looking for functions, narrowing down possible options based on behaviour
### shortlists made here will be tested further.


# nexts = []
# adds = []
ORS = []

for SKI_exp in SKI_data:
        
    try:
        
        maybe = SKIsimplify('((' + SKI_exp + ' * I) * I)')[0]
        if maybe == '((S * ((S * (K * S)) * K)) * I)':
            adds.append(SKI_exp)

        
        TT, stab1 = SKIsimplify("(("+ SKI_exp + " * K) * K)")
        FF, stab2 = SKIsimplify("(("+ SKI_exp + " * (K * I)) * (K * I))")
        TF, stab3 = SKIsimplify("(("+ SKI_exp + " * K) * (K * I))")
        FT, stab4 = SKIsimplify("(("+ SKI_exp + " * (K * I)) * K)")
        
        if TT == "K" and FF == "(K * I)" and TF == "K" and FT == "K":
                ORS.append(SKI_exp)
            
    except:
        pass

In [ ]:
def posINT_show(f):
    return f(lambda x: x + 1)(0)

posINT_show(S(S(K(S))(K))(ONE))

In [ ]:
def bool_SHOW(f):
    if f("honey")("badger") == "badger":
        return "FALSE"
        
    elif f("honey")("badger") == "honey":
        return "TRUE"
    
    else:
        return "faulty"

for OR in ORS:
    print(OR, "-----------", bool_SHOW(eval(OR)(TRUE)(TRUE)), bool_SHOW(eval(OR)(FALSE)(TRUE)), 
          bool_SHOW(eval(OR)(TRUE)(FALSE)), bool_SHOW(eval(OR)(FALSE)(FALSE)))

In [ ]:
print(adds[-1])

for add in adds:
    print(posINT_show(eval(SKIsimplify('((' + add + ' * I) * I)')[0])), posINT_show(eval(add)(TWO)(TWO)), "   ", add)

In [ ]:
ADD_string = '((S * I) * (K * (S * ((S * (K * S)) * K))))'
posINT_show(eval(ADD_string)(TWO)(ONE))
THREE = eval(ADD_string)(TWO)(ONE)
FOUR = eval(ADD_string)(TWO)(TWO)
FIVE = eval(ADD_string)(TWO)(THREE)
NINE = eval(ADD_string)(FOUR)(FIVE)
print(posINT_show(NINE))

In [ ]:
SKIsimplify('((' + ADD_string + ' * ((S * ((S * (K * S)) * K)) * ((S * ((S * (K * S)) * K)) * I))) * ((S * ((S * (K * S)) * K)) * ((S * ((S * (K * S)) * K)) * I)))')

In [ ]:
print(posINT_show(eval('((S * ((S * (K * S)) * K)) * ((S * ((S * (K * S)) * K)) * I))')))
print(posINT_show(eval('((S * ((S * (K * S)) * K)) * I)')))
print(posINT_show(eval('((S * ((S * (K * S)) * K)) * ((S * ((S * (K * S)) * K)) * ((S * ((S * (K * S)) * K)) * ((S * ((S * (K * S)) * K)) * ((S * ((S * (K * S)) * K)) * I)))))')))

posINT_show(eval(ADD_string)(THREE)(NINE))

In [ ]:
print(len(nexts))
real_nexts = []

for n in nexts:
    try:
        if posINT_show(eval(n)(ONE)) == 2:
            simple_n = SKIsimplify(n)[0]
            real_nexts.append(simple_n)
        
    except:
        pass

real_nexts = list(set(real_nexts))
print(sorted(real_nexts, key=len))

In [130]:
# Testing NEXT

NEXT_string = '(S * ((S * (K * S)) * K))'
print(posINT_show(eval(NEXT_string)(eval(NEXT_string)(ONE))))
THREE = eval(NEXT_string)(eval(NEXT_string)(ONE))
THREE = (S((S(K(S)))(K)))((S((S(K(S)))(K)))(I))

posINT_show(THREE)

simple_three, stab = SKIsimplify('(' + NEXT_string + ' * (' + NEXT_string + ' * I))')
print(simple_three, stab)

3
((S * ((S * (K * S)) * K)) * ((S * ((S * (K * S)) * K)) * I)) con


In [ ]:
print(posINT_show(S(I)(K(S((S(K(S)))(K))))(FOUR)(NINE)))

posINT_show(eval("((S * ((S * (K * S)) * K)) * I)"))

In [ ]:
SKI = "((S * (K * I)) * ((S * S) * K))"
# S(K(I))(S(S)(K))

print(bool_SHOW(eval(SKIsimplify("((" + SKI + " * K) * K)")[0])))
print(bool_SHOW(S(K(I))(S(S)(K))(TRUE)(TRUE)))

In [ ]:
# I = S(K)(K)
# FALSE = K(I)

# NOT = S(S(I)(K(K(I))))(K(K))

# OR = S(I)(K(K))
# AND = S(S)(K(K(K(I))))

#### These are backed up by [5] page 48.
# ONE = I
# NEXT = S((S(K(S)))(K))
# TWO = S((S(K(S)))(K))(I)
# THREE = S((S(K(S)))(K))((S((S(K(S)))(K)))(I))
# .
# .
# .

# ADD = S(I)(K(S((S(K(S)))(K))))
